# Pain Detection Models for SymPath

### Import required libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import pickle
import os
import sys
import librosa
import seaborn as sns
import keras
import warnings
import sounddevice as sd
from scipy.io.wavfile import write
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

Using TensorFlow backend.


# Facial Metrics Model

### Load dataset

In [2]:
data = pd.read_csv('facial_metrics_dataset.csv')

In [3]:
data.head()

,Unnamed: 0,joy,sadness,disgust,contempt,anger,fear,surprise,valence,engagement,...,smirk,eyeClosure,attention,lidTighten,jawDrop,dimpler,eyeWiden,cheekRaise,lipStretch,pain
0,0,100,0,0,0,0,0,1,91,100,...,0,0,92,0,1,0,0,1,5,0
1,1,5,0,0,0,0,0,0,42,100,...,12,0,94,10,0,0,0,1,1,1
2,2,0,0,0,0,0,0,0,0,0,...,0,0,96,0,0,0,0,0,0,0
3,3,100,0,0,0,0,0,2,47,100,...,0,6,99,0,1,0,0,87,0,0
4,4,0,0,0,0,0,0,1,-21,3,...,0,0,70,0,0,15,0,0,3,0


In [4]:
X = data.iloc[:, :-2].values
y = data.iloc[:, -1].values

In [5]:
X.shape, '  ', y.shape

((1202, 30), '  ', (1202,))

### Make train and test data and build model

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, ' ', X_test.shape, ' ', y_train.shape, ' ', y_test.shape

((961, 30), ' ', (241, 30), ' ', (961,), ' ', (241,))

In [7]:
model = RandomForestClassifier(n_estimators=20, random_state=0)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=20, random_state=0)

In [8]:
y_pred = model.predict(X_test)

In [9]:
threshold = 0.5
y_pred = y_pred > threshold

In [10]:
print('Accuracy Score:', metrics.accuracy_score(y_test, y_pred))

Accuracy Score: 0.91701244813278


### Save & reload model to test on new data

In [11]:
filename = 'facial_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [12]:
loaded_model = pickle.load(open(filename, 'rb'))
y_pred2 = loaded_model.predict(X_test)
threshold = 0.5
y_pred2 = y_pred2 > threshold
print('Accuracy Score:', metrics.accuracy_score(y_test, y_pred2))

Accuracy Score: 0.91701244813278


In [13]:
test_sample = X_test[10].reshape(1, -1)
single_prediction = loaded_model.predict(test_sample)

#### Close to 1 means more pain

In [14]:
"Predicted: " + str(single_prediction[0]) +  "   True: " + str(y_test[10])

'Predicted: 0.15   True: 0'

# Audio Emotion Model

### Feature extraction function used on training data

In [15]:
sample_rate = 22050

def extract_features(data):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.1)
    
    # without augmentation
    res1 = extract_features(data)
    result = np.array(res1)
    
    return result

### Loading model and indexed labels

In [16]:
labels = ['angry', 'calm', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [17]:
model = keras.models.load_model('audio_model.h5')

### Predict on data from microphone

In [18]:
def predict_audio(path):
    x = get_features(path)
    x = x.reshape(1, -1)
    x = np.expand_dims(x, 2)
    yp = model.predict(x)[0]
    print(yp[0]+yp[3]+yp[6]) # angry+fear+sad doesn't say much
    return labels[np.argmax(yp)]

#### Start speaking after you run this cell, it will record for 2.5 seconds

In [19]:
fs = sample_rate 
seconds = 2.5 # Duration of recording

myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
sd.wait()  # Wait until recording is finished
write('output.wav', fs, myrecording)  # Save as WAV file 

b = predict_audio('output.wav')
b

0.53221977


'angry'